In [1]:
home = "/home/paola/Family1070/private_output/ASE_rna/pipeline/"
setwd(home)

source("/frazer01/home/matteo/my_software/cardips_functions.R") 
source("/frazer01/home/paola/my_software/ase_pipeline_functions.R")    

rna_pipeline_dir_sample = "/frazer01/projects/CARDIPS/pipeline/RNAseq/sample"

In [2]:
sample_table = read.csv("../fam1070_data_plus_production-v3.csv", stringsAsFactors = FALSE) 
sample_table= subset(sample_table, Data_type=="RNA-Seq")
sample_table$Tissue = sample_table$Cell_type
sample_table$subject = sample_table$Subject


In [3]:
table(sample_table$Data_type, sample_table$Cell_type)

         
          iPSC iPSC-CM
  RNA-Seq   29      27

In [4]:
sample_table_full= sample_table

In [5]:
data_to_add = c( '142e5ecb-d354-4b16-9d4e-ee0110fd07d8','68db8f6f-5e8a-4354-babb-9515ae48df2a',
                 'abfc59bf-9fec-4a49-8855-93f27530772f')

In [6]:
sample_table = sample_table[sample_table$UUID %in% data_to_add,]

In [7]:
source("/frazer01/home/paola/my_software/ase_pipeline_functions.R")    
#####################################################################################
### change the queque type ### 
#####################################################################################
# writeHeader = function(id, folder, log_folder, ppn)
# {
#     header = paste("#!/bin/bash",
#                 "",
#                 paste("#$ -N job.", id, sep = ""),
#                 paste("#$ -o ", log_folder, "/", id, ".out", sep = ""),
#                 paste("#$ -e ", log_folder, "/", id, ".err", sep = ""),
#                 #"#$ -l all",
#                 paste("#$ -pe smp", ppn),
#                  "export PATH=/frazer01/home/paola/anaconda2/bin:$PATH",
# 				 "export PATH=/frazer01/home/paola/anaconda2/bin/python:$PATH",
               
# 				"module load cardips",
# 				"source activate cardips",
#                 sep = "\n"
#                )
			   
# 	outdir  = paste(folder, "wasp", sep = "/")
	
#     create_outdir = paste(paste("mkdir -p", outdir),
#                           paste("cd"      , outdir),
#                           sep = "\n"
#                          )

# 	out = paste(header, 
# 				create_outdir,
# 				sep = "\n\n"
# 			   )
			   
#     return(out);
# }
#####################################################################################
ppn        = 12
vcf_phased = "/projects/CARDIPS/analysis/family1070/sevenIndi.phased.variant.vcf.gz"
bed        = "/frazer01/publicdata/gencode_v19_20151104/genes.bed"

analysis_dir = paste(home, "analysis", sep = "/")
sh_dir       = paste(home, "sh"      , sep = "/")
log_dir      = paste(home, "log"     , sep = "/")


dir.create(analysis_dir, showWarnings = FALSE)
dir.create(sh_dir      , showWarnings = FALSE)
dir.create(log_dir     , showWarnings = FALSE)


for (tissue in sort(unique(sample_table$Tissue)))
{
    cat("Tissue : ", tissue, "\n")
    
    this_tissue = sample_table[sample_table$Tissue == tissue,]
    tissue_dir  = paste(analysis_dir, tissue, sep = "/")
    
    dir.create(tissue_dir, showWarnings = FALSE)

     for (name in sort(unique(this_tissue$UDID)))     ##### MODIFIED IN ORDER NOT TO MERGE REPLICATES OF SAME SUBJECT
    {
        cat("Name:", name, "\n")
        
       
        this_subject = this_tissue[this_tissue$UDID == name,]
        subject      = unique(this_subject$subject)
        subject_dir  = paste(tissue_dir, name,          sep = "/")
        full_name    = paste(tissue    , name, "genes", sep = "_")
        out_bam      = paste(subject_dir, "/", subject, ".bam", sep = "")
        bam_files    = paste(rna_pipeline_dir_sample, "/", this_subject$UUID, "/alignment/", this_subject$UUID, "_sorted_mdup.bam", sep = "")
        
        dir.create(subject_dir, showWarnings = FALSE)
        
        sh_file = runAse(full_name, sh_dir, subject, log_dir, subject, subject_dir, bam_files, out_bam, vcf_phased, bed, ppn, is.gz = TRUE, run = TRUE)
       
        # sh_file = runMbased(full_name, sh_dir, subject, log_dir, subject, subject_dir, bam_files, out_bam, vcf_phased, bed, ppn, is.gz = TRUE, run = TRUE)
    }
}


Tissue :  iPSC 
Name: FS027 
Name: FS029 
Name: FS040 


Run time ~ 3 hours

### After the previous analysis is done: merge data into tables

In [8]:
analysis_dir = paste(home, "analysis", sep = "")

In [9]:
sample_table = sample_table_full

In [12]:
pipeline = "/home/paola/Family1070/private_output/ASE_rna/pipeline/"
setwd(pipeline)
results_dir  = paste(pipeline, "results"     , sep = "")
dir.create(results_dir)

Warning message in dir.create(results_dir):
"'/home/paola/Family1070/private_output/ASE_rna/pipeline/results' already exists"

In [14]:
started_analysis = 0
mark= 'RNA-Seq' 
for (tissue in sort(unique(sample_table$Tissue)))
{
    cat("Tissue : ", tissue, "\n")
    this_tissue = sample_table[sample_table$Tissue == tissue,]
    tissue_dir  = paste(analysis_dir, tissue, sep = "/")

         for (name in sort(unique(this_tissue$UDID)))     ##### MODIFIED IN ORDER NOT TO MERGE REPLICATES OF SAME SUBJECT
    {
        cat("Name:", name, "\n")
            
            mbased_folder = paste(tissue_dir, name, "mbased", sep = "/")
            
        this_subject = this_tissue[this_tissue$UDID == name,]
        subject      = unique(this_subject$subject)
             
             
            locus_file    = paste(mbased_folder, "/", subject, "_locus.tsv"       , sep = "" )
            snv_file      = paste(mbased_folder, "/", subject, "_snv.tsv"         , sep = "" )
            in_file       = paste(mbased_folder, "/", subject, "_mbased_input.tsv", sep = "" )
            
            if ((file.exists(locus_file)== TRUE)&(file.exists(snv_file)== TRUE))
            {
                locus = read.table(locus_file, header = TRUE , sep = "\t", stringsAsFactors = FALSE, row.names = 1, comment.char = "") 
                snv   = read.table(snv_file  , header = TRUE , sep = "\t", stringsAsFactors = FALSE, row.names = 1, comment.char = "") 
                inp   = read.table(in_file   , header = TRUE , sep = "\t", stringsAsFactors = FALSE, row.names = 1, comment.char = "") 
                
                snv$ref_freq = snv$maf
                snv[snv$ref_is_major == FALSE, "ref_freq"] = 1 - snv[snv$ref_is_major == FALSE, "maf"]
                snv$coord   = paste(snv$chrom, snv$position, sep = ":")
                inp$coord   = paste(inp$contig, inp$position, sep = ":")

                locus$locus = rownames(locus)
                snv$maf = NULL

                out_data          = merge(locus, snv)
                out_data          = merge(inp[, c("coord", "variantID","refAllele", "altAllele",  "expectedRefFreq", "binomialPValue",  'refCount','altCount',"rawDepth")], out_data)
                out_data$subject  = subject
                out_data$UDID     = name
                out_data$mark     = mark
                out_data$tissue   = tissue
               
                
                if (started_analysis == 0)
                {
                    all_ase = out_data
                    started_analysis = 1
                    col_names_locus = colnames(locus)
                }else
                {
                    all_ase = rbind(all_ase, out_data)
                }
            }else
            {
                cat(locus_file, "does not exist\n")
            }
        }
        #break
 }

Tissue :  iPSC 
Name: 2_1.C4.P12 
Name: 2_2.C7.P13 
Name: 2_3.C5.P13 
Name: 2_4.C2.P13 
Name: 2_6.C1.P12 
Name: 2_7.C3.P12 
Name: 2_9.C5.P12 
Name: FS026 
Name: FS027 
Name: FS028 
Name: FS029 
Name: FS030 
Name: FS031 
Name: FS033 
Name: FS034 
Name: FS035 
Name: FS036 
Name: FS037 
Name: FS040 
Name: FS042 
Name: FS043 
Name: UDID007 
Name: UDID020 
Name: UDID043 
Name: UDID044 
Name: UDID045 
Name: UDID048 
Name: UDID053 
Name: UDID067 
Tissue :  iPSC-CM 
Name: FS003 
Name: FS005 
Name: FS006 
Name: FS007 
Name: FS008 
Name: FS009 
Name: FS010 
Name: FS011 
Name: FS014 
Name: FS015 
Name: FS016 
Name: FS017 
Name: FS018 
Name: FS024 
Name: UDID001 
Name: UDID007 
Name: UDID020 
Name: UDID021 
Name: UDID029 
Name: UDID034 
Name: UDID043 
Name: UDID044 
Name: UDID045 
Name: UDID048 
Name: UDID053 
Name: UDID067 


In [15]:
write.table(all_ase, file = paste(results_dir, "all_ase.txt", sep = "/"), col.names = TRUE, row.names = FALSE, quote = FALSE, sep = "\t")

In [16]:
sample_table$Mark="RNA-Seq"

In [ ]:
all_ase  = read.table(paste(results_dir, "all_ase.txt", sep = "/"), header = TRUE , sep = "\t", stringsAsFactors = FALSE) 

for (mark in sort(unique(sample_table$Mark)))
    {
        cat("Mark   : ", mark, "\n")
        this_mark = sample_table[sample_table$Mark == mark,]
        
        
    for (tissue in sort(unique(this_mark$Tissue)))
{
    cat("Tissue : ", tissue, "\n")
    this_tissue = this_mark[this_mark$Tissue == tissue,]
    tissue_dir  = paste(analysis_dir, tissue, sep = "/")
        
        separate_table = all_ase[all_ase$tissue == tissue & all_ase$mark == mark, ]
        
        this = unique(all_ase[all_ase$tissue == tissue & all_ase$mark == mark, c(col_names_locus, "UDID")])##### MODIFIED
        
        out_root = paste(results_dir, "/", tissue, "_", mark, sep = "")
        
        loci     = sort(unique(this$locus  ))
        subjects = sort(unique(this$UDID))  ##### MODIFIED
        
        empty = matrix(nrow = length(loci), ncol = length(subjects))
        rownames(empty) = loci
        colnames(empty) = subjects
        
        empty_df = data.frame(empty, stringsAsFactors = FALSE)
        
        p_val_ases = empty_df
        p_val_hets = empty_df
        mafs       = empty_df
        
        for (ii in 1: length(this$locus))
        {
            locus     = this$locus            [[ii]]
            subject   = this$UDID          [[ii]] ##### MODIFIED
            p_val_ase = this$p_val_ase        [[ii]]
            p_val_het = this$p_val_het        [[ii]]
            maf       = this$major_allele_freq[[ii]]
            
            p_val_ases[locus, subject] = p_val_ase
            p_val_hets[locus, subject] = p_val_het
            mafs      [locus, subject] = maf
        }
        write.table(p_val_ases, file = paste(out_root, "p_val_ase.txt", sep = "."), col.names = NA, row.names = TRUE, quote = FALSE, sep = "\t")
        write.table(p_val_hets, file = paste(out_root, "p_val_het.txt", sep = "."), col.names = NA, row.names = TRUE, quote = FALSE, sep = "\t")
        write.table(mafs      , file = paste(out_root, "major_allele_freq.txt"      , sep = "."), col.names = NA, row.names = TRUE, quote = FALSE, sep = "\t")
        write.table(separate_table, file = paste(out_root, "_ase_snv.txt", sep = ""), col.names = TRUE, row.names = FALSE, quote = FALSE, sep = "\t")               
 
    }
}
